In [87]:
import pandas as pd
import numpy as np
import yfinance as yf
from implied_vol import *
from finite_diff_methods import *

In [105]:
# download
AAPL = yf.Ticker("AAPL")
equity_data = spy.history('1Y')
expiry = ['2021-04-16', '2021-05-21', '2021-06-18']
call = []
put = []
for date in expiry:
    call.append(spy.option_chain(date)[0])
    put.append(spy.option_chain(date)[1])

In [116]:
pd.to_pickle(call, "./datasets/call.pkl")
pd.to_pickle(put, "./datasets/put.pkl")

In [107]:
# clean
def clean(data):
    new_data = []
    for df,date in zip(data, expiry):
        df['expiry'] = pd.to_datetime(date)
        df['t2m'] = (df['expiry'] - pd.Timestamp('today')) / np.timedelta64(1, 'Y')
        df['s0'] = equity_data.iloc[0,3]
        df['price'] = df.bid/2 + df.ask/2
        # choose by volume
        df = df.sort_values(by='lastTradeDate', ascending=False)
        new_df = df.iloc[0:10].reset_index()
        del new_df['index']
        new_data.append(new_df)
    return new_data
call = clean(call)
put = clean(put)

In [109]:
call[1]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiry,t2m,s0,price
0,SPY210521C00395000,2021-03-18 20:14:58,395.0,9.38,0.0,0.0,0.0,0.0,2996.0,0,0.003916,False,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
1,SPY210521C00400000,2021-03-18 20:14:42,400.0,6.95,0.0,0.0,0.0,0.0,1650.0,0,0.015635,False,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
2,SPY210521C00399000,2021-03-18 20:13:29,399.0,7.49,0.0,0.0,0.0,0.0,596.0,0,0.007822,False,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
3,SPY210521C00420000,2021-03-18 20:11:50,420.0,1.52,0.0,0.0,0.0,0.0,849.0,0,0.031260,False,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
4,SPY210521C00425000,2021-03-18 20:11:24,425.0,1.00,0.0,0.0,0.0,0.0,517.0,0,0.031260,False,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
5,SPY210521C00410000,2021-03-18 20:11:24,410.0,3.41,0.0,0.0,0.0,0.0,3695.0,0,0.031260,False,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
6,SPY210521C00359000,2021-03-18 20:08:47,359.0,36.87,0.0,0.0,0.0,0.0,32.0,0,0.000010,True,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
7,SPY210521C00392000,2021-03-18 20:08:38,392.0,11.39,0.0,0.0,0.0,0.0,419.0,0,0.000987,False,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
8,SPY210521C00384000,2021-03-18 20:08:02,384.0,16.71,0.0,0.0,0.0,0.0,406.0,0,0.000010,True,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0
9,SPY210521C00394000,2021-03-18 20:08:02,394.0,10.22,0.0,0.0,0.0,0.0,372.0,0,0.003916,False,REGULAR,USD,2021-05-21,0.172063,236.100845,0.0


In [67]:
# get vol
for df in call:
    call['vol'] = call.apply(lambda x: \
                             get_impliedVol('c', x.s0, x.strike, x.t2m, r, ))

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,s0
0,SPY210416C00210000,2021-02-26 20:53:31,210.0,173.32,0.00,0.00,0.0,0.0,1.0,0,0.00001,True,REGULAR,USD,236.100845
1,SPY210416C00220000,2021-02-03 14:47:57,220.0,171.37,162.99,164.31,0.0,0.0,5.0,0,0.00001,True,REGULAR,USD,236.100845
2,SPY210416C00225000,2021-01-11 15:11:27,225.0,154.74,165.45,166.02,0.0,0.0,1.0,0,0.00001,True,REGULAR,USD,236.100845
3,SPY210416C00230000,2021-03-15 14:33:10,230.0,163.60,0.00,0.00,0.0,0.0,33.0,0,0.00001,True,REGULAR,USD,236.100845
4,SPY210416C00240000,2021-03-11 17:03:34,240.0,153.44,0.00,0.00,0.0,0.0,3.0,0,0.00001,True,REGULAR,USD,236.100845
5,SPY210416C00245000,2020-10-29 14:32:50,245.0,88.49,118.39,120.04,0.0,0.0,NaN,5,0.00001,True,REGULAR,USD,236.100845
6,SPY210416C00250000,2021-03-05 16:58:51,250.0,147.50,0.00,0.00,0.0,0.0,4.0,0,0.00001,True,REGULAR,USD,236.100845
7,SPY210416C00260000,2021-03-05 19:35:50,260.0,122.70,0.00,0.00,0.0,0.0,2.0,0,0.00001,True,REGULAR,USD,236.100845
8,SPY210416C00265000,2021-02-05 20:47:43,265.0,123.14,118.48,119.06,0.0,0.0,1.0,30,0.00001,True,REGULAR,USD,236.100845
9,SPY210416C00270000,2021-03-17 19:32:07,270.0,127.45,0.00,0.00,0.0,0.0,1.0,0,0.00001,True,REGULAR,USD,236.100845
